## OpenMVG (Open Multiple View Geometry)
OpenMVG (Open Multiple View Geometry) is a C++ library.
https://github.com/openMVG

This script shows how to run commands through python using the subprocess module


In [5]:
# Indicate the openMVG binary directory.  This will depend on where you installed the code
# BUILD instructions are at https://github.com/openMVG/openMVG/blob/master/BUILD.md

OPENMVG_SFM_BIN = "/usr/local/bin/"

import os
import subprocess
import sys


In [3]:
# The dataset we will use comes from https://github.com/openMVG/ImageDataset_SceauxCastle.git
input_dir = os.path.relpath("data/images")
output_dir = os.path.relpath("data/tutorial_out")
matches_dir = os.path.relpath("data/tutorial_out/Matches")
camera_file_params = os.path.relpath("data/sensor_width_camera_database.txt")


In [6]:
print ("Step 1. Intrinsics analysis")
pIntrisics = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_SfMInit_ImageListing"),  "-i", input_dir, "-o", matches_dir, "-d", camera_file_params, "-c", "3"] )
pIntrisics.wait()


Step 1. Intrinsics analysis


INFO: [main_SfMInit_ImageListing.cpp:194]  You called : /usr/local/bin/openMVG_main_SfMInit_ImageListing
--imageDirectory data/images
--sensorWidthDatabase data/sensor_width_camera_database.txt
--outputDirectory data/tutorial_out/Matches
--focal -1
--intrinsics 
--camera_model 3
--group_camera_model 1
--use_pose_prior 0
--prior_weights 1.0;1.0;1.0
--gps_to_xyz_method 0
INFO: [loggerprogress.hpp:79] [- Listing images -] 100%
K.txt: Unkown image file format.
Readme.txt: Unkown image file format.

INFO: [main_SfMInit_ImageListing.cpp:475] SfMInit_ImageListing report:
listed #File(s): 13
usable #File(s) listed in sfm_data: 11
usable #Intrinsic(s) listed in sfm_data: 1


0

In [7]:
print ("Step 2. Compute features")
pFeatures = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_ComputeFeatures"),  "-i", matches_dir+"/sfm_data.json", "-o", matches_dir, "-m", "SIFT", "-f" , "1"] )
pFeatures.wait()

Step 2. Compute features


INFO: [main_ComputeFeatures.cpp:120]  You called : 
/usr/local/bin/openMVG_main_ComputeFeatures
--input_file data/tutorial_out/Matches/sfm_data.json
--outdir data/tutorial_out/Matches
--describerMethod SIFT
--upright 0
--describerPreset NORMAL
--force 1

INFO: [loggerprogress.hpp:79] [- EXTRACT FEATURES -] 100%
INFO: [main_ComputeFeatures.cpp:343] Task done in (s): 451


0

In [8]:
print ("Step 3. Compute matches")
pMatches = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_ComputeMatches"),  "-i", matches_dir+"/sfm_data.json", "-o", matches_dir+"/matches.putative.bin", "-f", "1", "-n", "ANNL2"] )
pMatches.wait()

Step 3. Compute matches


INFO: [main_ComputeMatches.cpp:113]  You called : 
/usr/local/bin/openMVG_main_ComputeMatches
--input_file data/tutorial_out/Matches/sfm_data.json
--output_file data/tutorial_out/Matches/matches.putative.bin
--pair_list 
Optional parameters:
--force 1
--ratio 0.8
--nearest_matching_method ANNL2
--cache_size unlimited
--preemptive_feature_used/count 0 / 200
INFO: [loggerprogress.hpp:79] [- Regions Loading -] 100%
INFO: [main_ComputeMatches.cpp:216]  - PUTATIVE MATCHES - 
INFO: [main_ComputeMatches.cpp:279] Using ANN_L2 matcher
INFO: [main_ComputeMatches.cpp:306] No input pair file set. Use exhaustive match by default.
INFO: [main_ComputeMatches.cpp:316] Running matching on #pairs: 55
INFO: [loggerprogress.hpp:79] [- Matching -] 20%
INFO: [loggerprogress.hpp:79] [- Matching -] 40%
INFO: [loggerprogress.hpp:79] [- Matching -] 60%
INFO: [loggerprogress.hpp:79] [- Matching -] 80%
INFO: [loggerprogress.hpp:79] [- Matching -] 100%
INFO: [main_ComputeMatches.cpp:362] Task (Regions Matching) do

0

# Reconstruction using the sequential method

In [9]:
print ("Step 4a. Filter matches" )
pFiltering = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_GeometricFilter"), "-i", matches_dir+"/sfm_data.json", "-m", matches_dir+"/matches.putative.bin" , "-g" , "f" , "-o" , matches_dir+"/matches.f.bin" ] )
pFiltering.wait()

Step 4a. Filter matches


INFO: [main_GeometricFilter.cpp:131]  You called : 
/usr/local/bin/openMVG_main_GeometricFilter
--input_file:        data/tutorial_out/Matches/sfm_data.json
--matches:           data/tutorial_out/Matches/matches.putative.bin
--output_file:       data/tutorial_out/Matches/matches.f.bin
Optional parameters: 
--input_pairs        
--output_pairs       
--force              false
--geometric_model    f
--guided_matching    0
--cache_size         unlimited
INFO: [loggerprogress.hpp:79] [- Regions Loading -] 100%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 20%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 40%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 60%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 80%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 100%
INFO: [graph_stats.hpp:53] Graph statistics:
	#nodes: 11
	#cc: 1
	#singleton: 0
	Node degree statistics:	min: 10, max: 10, mean: 10, median: 10
INFO: [main_GeometricFilter.cpp:383] Task 

0

In [10]:
reconstruction_dir = os.path.join(output_dir,"reconstruction_sequential")
print ("Step 5a. Do Incremental/Sequential reconstruction") #set manually the initial pair to avoid the prompt question
pRecons = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_SfM"), "--sfm_engine", "INCREMENTAL", "--input_file", matches_dir+"/sfm_data.json", "--match_dir", matches_dir, "--output_dir", reconstruction_dir] )
pRecons.wait()

Step 5a. Do Incremental/Sequential reconstruction


INFO: [main_SfM.cpp:157] 
-----------------------------------------------------------
 Structure from Motion:
-----------------------------------------------------------
INFO: [loggerprogress.hpp:79] [- Features Loading -] 100%
INFO: [sequential_SfM.cpp:278] Track building
INFO: [sequential_SfM.cpp:281] Track filtering
INFO: [sequential_SfM.cpp:283] Track export to internal struct
INFO: [sequential_SfM.cpp:309] 
------------------
-- Tracks Stats --
 Tracks number: 6582
 Images Id: 
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
------------------
TrackLength, Occurrence
	2	3072
	3	1297
	4	717
	5	477
	6	276
	7	254
	8	233
	9	157
	10	77
	11	22

INFO: [loggerprogress.hpp:79] [Selection of an initial pair] 20%
INFO: [loggerprogress.hpp:79] [Selection of an initial pair] 40%
INFO: [loggerprogress.hpp:79] [Selection of an initial pair] 60%
INFO: [loggerprogress.hpp:79] [Selection of an initial pair] 80%
INFO: [loggerprogress.hpp:79] [Selection of an initial pair] 100%
INFO: [sequential_SfM.cpp:495] Puta

0

In [11]:
print ("6a. Colorize Structure")
pRecons = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_ComputeSfM_DataColor"),  "-i", reconstruction_dir+"/sfm_data.bin", "-o", os.path.join(reconstruction_dir,"colorized.ply")] )
pRecons.wait()


6a. Colorize Structure


INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 10%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 20%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 30%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 40%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 50%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 60%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 70%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 80%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 90%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 100%


0

In [12]:
print ("7a. Structure from Known Poses (robust triangulation)")
pRecons = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_ComputeStructureFromKnownPoses"),  "-i", reconstruction_dir+"/sfm_data.bin", "-m", matches_dir, "-o", os.path.join(reconstruction_dir,"robust.ply")] )
pRecons.wait()


7a. Structure from Known Poses (robust triangulation)


INFO: [main_ComputeStructureFromKnownPoses.cpp:56] Compute Structure from the provided poses
INFO: [main_ComputeStructureFromKnownPoses.cpp:134] Loaded a sfm_data scene with:
 #views: 11
 #poses: 11
 #intrinsics: 1
 #tracks: 0
INFO: [loggerprogress.hpp:79] [- Regions Loading -] 100%
INFO: [main_ComputeStructureFromKnownPoses.cpp:299] Robust triangulation of the tracks:  [Triangulation of guided epipolar geometry matches]
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 20%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 40%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 60%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 80%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 100%
INFO: [loggerprogress.hpp:79] [Pairwise fundamental guided matching] 20%
INFO: [loggerprogress.hpp:79] [Pairwise fundamental guided matching] 40%
INFO: [loggerprogress.hpp:79] [Pairwise fundamental guided matching] 60%
INFO: [loggerprogres

0

## Reconstruction for the global SfM pipeline
- global SfM pipeline use matches filtered by estimating essential matrices
- here we reuse photometric matches and perform only the essential matrix filering

In [13]:

print ("Step 4b. Filter matches (for the global SfM Pipeline)")
pFiltering = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_GeometricFilter"), "-i", matches_dir+"/sfm_data.json", "-m", matches_dir+"/matches.putative.bin" , "-g" , "e" , "-o" , matches_dir+"/matches.e.bin" ] )
pFiltering.wait()

Step 4b. Filter matches (for the global SfM Pipeline)


INFO: [main_GeometricFilter.cpp:131]  You called : 
/usr/local/bin/openMVG_main_GeometricFilter
--input_file:        data/tutorial_out/Matches/sfm_data.json
--matches:           data/tutorial_out/Matches/matches.putative.bin
--output_file:       data/tutorial_out/Matches/matches.e.bin
Optional parameters: 
--input_pairs        
--output_pairs       
--force              false
--geometric_model    e
--guided_matching    0
--cache_size         unlimited
INFO: [loggerprogress.hpp:79] [- Regions Loading -] 100%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 20%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 40%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 60%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 80%
INFO: [loggerprogress.hpp:79] [- Geometric filtering -] 100%
INFO: [graph_stats.hpp:53] Graph statistics:
	#nodes: 11
	#cc: 1
	#singleton: 0
	Node degree statistics:	min: 7, max: 10, mean: 9, median: 10
INFO: [main_GeometricFilter.cpp:383] Task do

0

In [14]:
reconstruction_dir = os.path.join(output_dir,"reconstruction_global")
print ("Step 5b. Do Global reconstruction")
pRecons = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_SfM"), "--sfm_engine", "GLOBAL", "--input_file", matches_dir+"/sfm_data.json", "--match_file", matches_dir+"/matches.e.bin", "--output_dir", reconstruction_dir] )
pRecons.wait()

Step 5b. Do Global reconstruction


INFO: [main_SfM.cpp:157] 
-----------------------------------------------------------
 Structure from Motion:
-----------------------------------------------------------
INFO: [loggerprogress.hpp:79] [- Features Loading -] 100%
INFO: [connectedComponent.hpp:99] CleanGraph_KeepLargestBiEdge_Nodes():: => connected Component: 1
INFO: [connectedComponent.hpp:118] Connected component of size: 11
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 10%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 20%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 30%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 40%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 50%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 60%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 70%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 80%
INFO: [loggerprogress.hpp:79] [- Relative pose computation -] 90%
INFO: [logge

0

In [15]:
print ("Step 6b. Colorize Structure")
pRecons = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_ComputeSfM_DataColor"),  "-i", reconstruction_dir+"/sfm_data.bin", "-o", os.path.join(reconstruction_dir,"colorized.ply")] )
pRecons.wait()

Step 6b. Colorize Structure


INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 10%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 20%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 30%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 40%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 50%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 60%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 70%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 80%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 90%
INFO: [loggerprogress.hpp:79] [- Compute scene structure color -] 100%


0

In [16]:
print ("Step 7b. Structure from Known Poses (robust triangulation)")
pRecons = subprocess.Popen( [os.path.join(OPENMVG_SFM_BIN, "openMVG_main_ComputeStructureFromKnownPoses"),  "-i", reconstruction_dir+"/sfm_data.bin", "-m", matches_dir, "-o", os.path.join(reconstruction_dir,"robust.ply")] )
pRecons.wait()

Step 7b. Structure from Known Poses (robust triangulation)


INFO: [main_ComputeStructureFromKnownPoses.cpp:56] Compute Structure from the provided poses
INFO: [main_ComputeStructureFromKnownPoses.cpp:134] Loaded a sfm_data scene with:
 #views: 11
 #poses: 11
 #intrinsics: 1
 #tracks: 0
INFO: [loggerprogress.hpp:79] [- Regions Loading -] 100%
INFO: [main_ComputeStructureFromKnownPoses.cpp:299] Robust triangulation of the tracks:  [Triangulation of guided epipolar geometry matches]
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 20%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 40%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 60%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 80%
INFO: [loggerprogress.hpp:79] [Computing frustum intersection] 100%
INFO: [loggerprogress.hpp:79] [Pairwise fundamental guided matching] 20%
INFO: [loggerprogress.hpp:79] [Pairwise fundamental guided matching] 40%
INFO: [loggerprogress.hpp:79] [Pairwise fundamental guided matching] 60%
INFO: [loggerprogres

0